In [7]:
import pandas as pd
import numpy as np

In [9]:
train_data = pd.read_csv("Gotem Pumpkins.csv")
test_data = pd.read_csv("Freyja_Pumpkins.csv")

print(train_data.head())
print("TEST DATA")
print(test_data.head())

   Unnamed: 0    Area  Perimeter  Major_Axis_Length  Minor_Axis_Length  \
0           0   84930   1204.448           508.7883           213.0266   
1           1  103343   1275.261           521.4694           252.8941   
2           2   98350   1269.539           526.2346           238.4702   
3           3   69517   1059.538           429.8407           206.8250   
4           4   80011   1182.947           501.9065           204.7531   

   Convex_Area  Equiv_Diameter  Eccentricity  Solidity  Extent  Roundness  \
0        85665        328.8407        0.9081    0.9914  0.5482     0.7357   
1       104156        362.7401        0.8745    0.9922  0.7457     0.7985   
2        99184        353.8688        0.8914    0.9916  0.5919     0.7668   
3        70493        297.5093        0.8766    0.9862  0.6623     0.7782   
4        80902        319.1758        0.9130    0.9890  0.7374     0.7185   

   Aspect_Ration  Compactness          Class  
0         2.3884       0.6463  Ürgüp Sivrisi 

## Extracting Features and Target Variable

In [10]:
X_train = train_data.drop(columns='Class').values
y_train = train_data['Class'].values

X_test = test_data.drop(columns='Class').values
y_test = test_data['Class'].values

## Normalization

In [11]:
X_train_min = X_train.min(axis=0)
X_train_max = X_train.max(axis=0)

X_train = (X_train - X_train_min) / (X_train_max - X_train_min)
X_test = (X_test - X_train_min) / (X_train_max - X_train_min)